In [1]:
from terrain_generation import generate_map
import json
from ipywidgets import interact
import ipywidgets as widgets

In [4]:
default_map_config = json.load(open("default_map_config.json"))

_ = interact(
    generate_map,
    plot=True,
    return_map_data=False,
    senders=widgets.IntSlider(min=1, max=20, step=0.1, value=1),
    receivers=widgets.IntSlider(min=1, max=20, step=0.1, value=10),
    map_resolution=widgets.IntSlider(min=100, max=1000, step=1, value=default_map_config["map_resolution"]),
    foliage_freq=widgets.FloatSlider(min=1, max=10, step=0.1, value=default_map_config["foliage_freq"]),
    foliage_octaves=widgets.IntSlider(min=1, max=10, step=1, value=default_map_config["foliage_octaves"]),
    foliage_octaves_factor=widgets.FloatSlider(min=1, max=10, step=0.1, value=default_map_config["foliage_octaves_factor"]),
    foliage_levels=widgets.IntSlider(min=0, max=10, step=0.1, value=default_map_config["foliage_levels"]),
    foliage_redistribution=widgets.FloatSlider(min=0, max=10, step=0.1, value=default_map_config["foliage_redistribution"]),
    pop_octaves=widgets.IntSlider(min=1, max=10, step=1, value=default_map_config["pop_octaves"]),
    pop_freq=widgets.FloatSlider(min=1, max=10, step=0.1, value=default_map_config["pop_freq"]),
    pop_octaves_factor=widgets.FloatSlider(min=1, max=3, step=0.01, value=default_map_config["pop_octaves_factor"]),
    pop_levels=widgets.IntSlider(min=2, max=20, step=0.1, value=default_map_config["pop_levels"]),
    pop_redistribution=widgets.FloatSlider(min=0, max=10, step=0.1, value=default_map_config["pop_redistribution"]),
)

interactive(children=(IntSlider(value=1, description='senders', max=20, min=1, step=0), IntSlider(value=10, de…